# A-gs model and implementation (simulation CO2 and H2O flux)

## Initialize data and model

### Setup and fetch data

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
from Ags_model import runAgs, calc_LE

In [ ]:
import matplotlib.dates as mdates

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')

In [ ]:
#alternate filtering
from FilterData import Filter_GPP_LE_NEE_VPD
df_Comb_alt_filter = Filter_GPP_LE_NEE_VPD(df_Comb,fqc=None)

In [ ]:
#cell used for inspecting data
#st='2008-01-01'
#ed='2019-12-30'
#df_meteo.loc[st:ed,'L(o)'].plot()
#df_meteo.loc[st:ed,'L(o)corr'].plot()
#df_meteo.loc[st:ed,'Te-L(o)'].plot()
#df_profile.loc[st:ed,'CO2level1'].plot()
#df_profile.loc[st:ed,'Pressure'].plot()
#df_Comb.loc[st:ed,'VPD'].plot()
#df_Comb.loc[st:ed,'Tair'].plot()
#df_Comb.loc[st:ed,'GPP_fqc'].plot()
#df_EC.loc[st:ed,'U-star'].plot()
#df_EC.loc[st:ed,'Mea_Windsp'].plot()

### Import rain data

In [ ]:
#datapath = os.path.join('../KNMI_regen')
#print(datapath)
knmi_regen=pd.read_csv('../KNMI_regen/neerslaggeg_KOOTWIJK-RADIO_567.txt',sep=',',header=17)
knmi_regen=knmi_regen.iloc[28000:] #remove dataset before ~november 2007
#knmi_regen=knmi_regen.iloc[31000:] #remove dataset before ~november 2007
knmi_regen['date']=pd.to_datetime(knmi_regen['YYYYMMDD'],yearfirst=True,format='%Y%m%d')
knmi_regen.index=knmi_regen['date']
knmi_regen=knmi_regen.rename(columns={"   RD": "RD", "   SX": "SX"}) #for some reason there's spaces in some of the column names, removing them.
knmi_regen=knmi_regen.astype({"RD": np.intc})
knmi_regen['RD'].plot() #Column RD is is rain daily sum, SX is code related to snow. If unnamed contains 4 spaces it's also a code for snow. STN is station ID.
df_rain=knmi_regen.loc[:,['RD']].resample('3H').ffill()
#df_rain

### Run A-gs model

In [ ]:
# Run A-gs model

an_final,an_umol,rs, ra = runAgs(df_profile_filter,df_Comb_filter,df_meteo_filter,df_EC_filter,fstr=1.0)

In [ ]:
#GPP_sim=an_umol.resample("M").mean()
#GPP_sim.plot(label="fstr=1.0",legend='yes',title="Monthly average GPP model output",ylabel="umolm-2s-1")
#GPP_meas=df_Comb_filter['GPP_f'].resample("M").mean()
#GPP_meas.plot(c='black', linestyle='dashed',label='measured',legend='yes')

In [ ]:
#rs.resample("M").mean().plot()

## Calcuate ET

### Assemble dataframe 'df_ET' that will hold output and fill with inputs

In [ ]:
def init_ETframe(rs_series):
    df_ET = pd.concat([df_meteo['L(o)'],df_meteo['Te-L(o)'],df_profile['Pressure'],df_Comb['VPD'],df_Comb['rH'],df_meteo['P(mast)']],axis=1,sort=False)
    #convert Pressure from hPa to kPa 
    df_ET['p_kPa']=df_ET['Pressure']/10
    df_ET['VPD_adj']=df_ET['VPD'].loc[df_ET['VPD']>0] #some outlier values for VPD are negative, remove from dataset
    df_ET['VPD_adj']=df_ET['VPD_adj']/10  # VPD from df_Comb is in hPa, I need kPa, so hPa/10 = kPa
    df_ET['rs']=rs_series.to_frame(name="rs")
    df_ET['ra']=ra.to_frame(name="ra")
    return df_ET

In [ ]:
df_ET=init_ETframe(rs)

In [ ]:
df_ET=calc_LE(df_ET)

In [ ]:
df_ET_meas=df_Comb_filter.copy()
df_ET_meas=df_ET_meas.loc[df_ET_meas['LE']>=0]
df_ET_meas=df_ET_meas.resample("M").mean()
#df_ET_meas['LE'].plot()

In [ ]:
#df_ET['ET_VPD'].resample('M').mean().plot(label="fstr=1.0",legend='yes',title="Monthly average LE model output",ylabel="Wm-2")
#df_ET_meas['LE'].resample('M').mean().plot(c='black', linestyle='dashed',label='measured',legend='yes')

In [ ]:
# Add in precipitation last 3h and 24 hour values so it can be used for filtering
df_ET['last3day_prec']=df_ET['P(mast)'].rolling('72H').sum()

### graphical check data for outliers

In [ ]:
df_Comb['LE'].plot()

In [ ]:
st="2017-06-01" 
ed="2017-12-30"
#df_meteo['L(o)corr'].loc[df_meteo['L(o)corr']>0].plot()
#df_meteo.loc[st:ed,'L(o)corr'].plot()
df_meteo['L(o)corr'].plot()

## resampling procedures

### step 1 assemble dataframe and chop up according to each week

In [ ]:
df_tmp30m=pd.DataFrame()
df_tmp30m['ET']=df_ET['ET_VPD']
df_tmp30m['P(mast)']=df_ET['P(mast)']
df_tmp30m=df_tmp30m.dropna()

df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)

#df_tmp30['ET/LE']=df_tmp30['ET']/df_tmp30['LE']
#df_tmp30m['LE/ET']=df_tmp30m['LE']/df_tmp30m['ET'] #<- this is the one we want, observations/model. If obs/model = fstr, then model*fstr= model(obs/model) = obs

df_tmp30m['P_3day']=df_tmp30m['P(mast)'].rolling('72H').sum()
#df_tmp30m=df_tmp30m.loc[df_tmp30m['P_3day']==0]

df_tmp3h=df_tmp30m.resample('3H').mean()#.between_time("09:00", "15:00")
df_tmp3h['P(mast)']=df_tmp30m['P(mast)'].resample('3H').sum() #overwrite the averaged pmast with sum pmast
df_tmp3h['P_3day']=df_tmp3h['P(mast)'].rolling('72H').sum() #overwrite the correct P_3day

#remove rainy days through knmi data
df_tmp3h['RD']=df_rain['RD']
df_tmp3h['RD']=df_tmp3h['RD']/10 #convert from tenths of mm to mm
df_tmp3h=df_tmp3h.loc[df_tmp3h['RD']==0]

#remove rainy days through Pmast
df_rain3h_dailymax=df_tmp3h['P(mast)'].resample('1D').sum().resample('3H').ffill()
df_rain3h_dailymax=df_rain3h_dailymax[:-1] #remove last entry because all 3h frames end on 21:00 not 00:00
df_rain3h_dailymax=df_rain3h_dailymax.rename('P(mast)Dmax')
df_tmp3h['P(mast)Dmax']=df_rain3h_dailymax
df_tmp3h=df_tmp3h.loc[df_tmp3h['P(mast)Dmax']==0]

#manual filter remove bad days
listofdates =  ['2008-08-24',
                '2010-05-11',
                '2010-08-04',
                '2012-06-13',
                '2013-05-10',
                '2013-05-19',
                '2014-06-16',
                '2017-04-22']

#for date in listofdates:
#    df_tmp3h.loc[date+' 00:00':date+' 21:00','ET'] = np.nan
#    df_tmp3h.loc[date+' 00:00':date+' 21:00','LE'] = np.nan

#remove hours outside of 9:00 and 15:00
#df_tmp3h=df_tmp3h.between_time("09:00", "15:00")

#resample to daily
df_tmp1d=df_tmp3h.resample('1D').mean()
df_tmp1d['doy']=df_tmp1d.index.dayofyear
df_tmp1d['P(mast)sum']=df_tmp3h['P(mast)'].resample('1D').sum()
df_tmp1d['LE/ET']=df_tmp1d['LE']/df_tmp1d['ET']

#resample to weekly (7 days)
df_tmp7d=df_tmp3h.resample('7D').mean()
df_tmp7d['LE/ET']=df_tmp7d['LE']/df_tmp7d['ET']

## plots of rain bars and fstr dots

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)

#st="2005-04-01"
#ed="2005-09-30"

def fstr_plot_wrapper(st,ed):

    fig,ax = plt.subplots()
    
    b=df_ET.loc[st:ed,['P(mast)']].resample('1D').sum()
    a=(knmi_regen.loc[st:ed,['RD']]/10)
    ax.bar(b.index,b['P(mast)'],alpha=0.5, align='edge', label="Pmast data")
    ax.bar(a.index,a['RD'],alpha=0.5, align='edge', label="KNMI data")
    
    df_tmp30m=pd.DataFrame()
    df_tmp30m['ET']=df_ET['ET_VPD']
    df_tmp30m['P(mast)']=df_ET['P(mast)']
    df_tmp30m=df_tmp30m.dropna()
    
    df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
    df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)
    c=df_tmp30m.loc[st:ed].resample('1D').sum()
    c['LE/ET']=c['LE']/c['ET']
    
    ax.set_ylabel("precip")
    ax2 = ax.twinx()
    ax2.set_ylim(0,5)
    ax2.set_ylabel("fstr")
    #ax2.set_yscale('log')
    
    ax2.scatter(c.index,c['LE/ET'],c=np.where(c['LE/ET']<1,'b','r'))
    
    ax.legend()
    ax.xaxis.set_minor_locator(mdates.DayLocator())

for year in range(2001,2021):
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    fstr_plot_wrapper(st,ed)

## end of plots of rain bars and fstr dots

### visualize each year to see whats going wrong

## plots of obs and model signal

### wrapper function def

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)
st="2011-04-01"
ed="2011-09-30"
#df_tmp.loc[st:ed,'LE/ET'].plot()

def signal_plot_wrapper(data,st,ed,gridarg=None,tick_frq='Day'):
    fig,ax = plt.subplots()
    fig.subplots_adjust(right=0.75)
    
    twin1 = ax.twinx()
    twin2 = ax.twinx()
    
    # Offset the twin axis below the host
    #fig.subplots_adjust(bottom=0.2)
    twin2.spines["right"].set_position(("axes", 1.05))
    twin1.set_ylabel("Precip 3day (Pmast) [mm]")
    twin2.set_ylabel("Precip knmi [tenth of mm]")

    if '30m' in data:
        ax.plot(df_tmp30m.loc[st:ed,'LE'],label="LE obs 30m freq")
        ax.plot(df_tmp30m.loc[st:ed,'ET'],label="LE model 30m freq")
    if '3h' in data:
        ax.plot(df_tmp3h.loc[st:ed,'LE'],label="LE obs 3H mean")
        ax.plot(df_tmp3h.loc[st:ed,'ET'],label="LE model 3H mean")
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'LE'],c='g')
        ax.scatter(df_tmp3h.loc[st:ed].index,df_tmp3h.loc[st:ed,'ET'],c='r')
    if '7d' in data:
        ax.plot(df_tmp7d.loc[st:ed,'LE'],c='#1f77b4',linestyle='--',label="LE obs 7day mean")
        ax.plot(df_tmp7d.loc[st:ed,'ET'],c='orange',linestyle='--',label="LE model 7day mean")

    twin1.plot(df_ET.loc[st:ed,'P(mast)'],c='red',label="Pmast")
    twin1.plot(df_ET.loc[st:ed,'last3day_prec'],c='black',label="Pmast 72h")
    twin2.bar(knmi_regen.loc[st:ed].index,knmi_regen.loc[st:ed,'RD'],width=1, align='edge', color="blue",alpha=0.5, label="KNMI Precip")[0]
    twin2.bar(df_tmp1d.loc[st:ed].index,df_tmp1d.loc[st:ed,'P(mast)sum'],width=1, align='edge', color="red",alpha=0.5, label="Pmast Precip")[0]
    
    if tick_frq=='Day':
        ax.xaxis.set_minor_locator(mdates.DayLocator())
    elif tick_frq=='Hour':
        ax.xaxis.set_minor_locator(mdates.HourLocator())
    
    ax.set_ylabel('LE [Wm-2]')
    
    ax.legend(loc=("upper left"))
    fig.autofmt_xdate()
    if gridarg==None:
        ax.grid()
    else:
        ax.grid(which=gridarg)
    #df_tmp30.loc[st:ed,'LE'].plot(label="LE obs 3H mean",legend="y",ylabel="Wm-2",title="LEobs and LEsim 3H mean and 7day mean values, 00:00-23:30")
    #df_tmp30.loc[st:ed,'ET'].plot(label="LE model 3H mean",legend="y")
    #df_tmp.loc[st:ed,'LE'].plot(label="LE obs 7day mean",legend="y",c='#1f77b4',style='--')
    #df_tmp.loc[st:ed,'ET'].plot(label="LE model 7day mean",legend="y",c='orange',style='--')
    #df_ET.loc[st:ed,'P(mast)'].plot()
    plt.show()

### all years 2001-2020 growseason (1 may - 1 october)

In [ ]:
for year in range(2001,2020):
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    signal_plot_wrapper(['3h','7d'],st,ed)

### zoom in on 2007

In [ ]:
for year in range(2007,2008): #2007 has a peculiar dry spring
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    signal_plot_wrapper(['3h','7d'],st,ed)

In [ ]:
for year in range(2007,2008): #zoom in on only the spring
    st=str(year)+"-04-01"
    ed=str(year)+"-05-09"
    signal_plot_wrapper(['30m','3h','7d'],st,ed)

### zoom in on 2018

In [ ]:
for year in range(2018,2019): #2018 has a very dry summer
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    signal_plot_wrapper(['3h','7d'],st,ed)

In [ ]:
for year in range(2018,2019): #zoom in on dry summer
    st=str(year)+"-06-21"
    ed=str(year)+"-08-15"
    signal_plot_wrapper(['30m','3h','7d'],st,ed)

In [ ]:
for year in range(2018,2019): #zoom in on june-july
    st=str(year)+"-06-23"
    ed=str(year)+"-07-10"
    signal_plot_wrapper(['30m','3h','7d'],st,ed)

In [ ]:
for year in range(2018,2019): #zoom in on june-july
    st=str(year)+"-06-25"
    ed=str(year)+"-06-29"
    signal_plot_wrapper(['30m','3h','7d'],st,ed,'both','Hour')

In [ ]:
for year in range(2018,2019): #zoom in for 48H
    st=str(year)+"-06-26"
    ed=str(year)+"-06-28"
    signal_plot_wrapper(['30m','3h','7d'],st,ed,'both','Hour')

In [ ]:
df_test=df_tmp30m.copy()
st=str(year)+"-06-26"
ed=str(year)+"-06-28"
df_test=df_test.loc[st:ed]
print(df_test.head(30))
df_test_3h=df_test.resample('3H').mean()
print(df_test_3h)

## end of plots of obs and model signal

## plot obs-sim timeseries for heatwave years with visualization

In [ ]:
heatwave=pd.read_csv('Heatwave_table.csv')
heatwave.loc[:,'start']        = pd.to_datetime(heatwave['start'])
heatwave.loc[:,'end(inc.)']    = pd.to_datetime(heatwave['end(inc.)'])
heatwave.loc[:,'MaxTempDate']  = pd.to_datetime(heatwave['MaxTempDate'])
heatwave

In [ ]:
dates=heatwave.loc[:,['start','end(inc.)']]

for index,row in dates.iterrows(): #zoom in for 48H
    #print(row['start'])
    #st=str(year)+"-06-26"
    #ed=str(year)+"-06-28"
    signal_plot_wrapper(['30m','3h','7d'],row['start'],row['end(inc.)'],'both','Hour')

## end of plot obs-sim timeseries for heatwave years with visualization

## special plots for heatwaves 2013 and 2015

In [ ]:
st='2013-07-03'
ed='2013-07-27'
signal_plot_wrapper(['30m','3h','7d'],st,ed,'both','Day')

In [ ]:
st='2013-07-03'
ed='2013-07-27'
signal_plot_wrapper(['30m','3h','7d'],st,ed,'both','Day')

In [ ]:
st='2015-06-23'
ed='2015-07-05'
signal_plot_wrapper(['30m','3h','7d'],st,ed,'both','Day')

## end of special plots for heatwaves 2013 and 2015

## completed special plots for heatwaves 2013 and 2015

### prepare data LE, H, Bowen for plotting

In [ ]:
#preparing the data
st='2013-06-01'
ed='2015-09-30'
#f_Comb.loc[st:ed,'LE_f'].plot() #this data is with gapfill . Label 'LE' is without
#df_NEE.loc[st:ed,'LE_f'].plot() #this is the same as the last line
LE_plotting = df_Comb.loc[df_Comb['LE_fqc']<=2].loc[st:ed,'LE_f'] #outputs a series
LE_plotting.where(LE_plotting>0,inplace=True) #filters that series so nonsensical negative evap is removed
H_plotting = df_Comb.loc[df_Comb['H_fqc']<=2].loc[st:ed,'H_f'] #outputs a series
H_plotting.where(H_plotting>0,inplace=True) #filters that series so nonsensical negative evap is removed
Bowen_plotting = df_ET = pd.concat([LE_plotting, H_plotting],axis=1,sort=False)
Bowen_plotting.dropna(inplace=True)
Bowen_plotting['Bowen']=Bowen_plotting['H_f']/Bowen_plotting['LE_f']

### end of prepare data LE, H, Bowen for plotting

### investigating gap in data by selecting less certain quality flags, look at Bowen

In [ ]:
st='2013-07-03'
ed='2013-07-27'
LE_plotting.plot()

In [ ]:
st='2013-07-03'
ed='2013-07-27'
H_plotting.plot()

In [ ]:
st='2013-07-03'
ed='2013-07-27'
Bowen_plotting.loc[st:ed,'Bowen'].plot()

In [ ]:
#Bowen ratio

### end of investigating gap in data

#### Import SM, Precip

In [ ]:
#data sources
#LE df_Comb, dont even need ot run filter
#H df_Comb, see previous line
#GPP (bonus, implement later) 
#SM from Soilmoisture.csv
#Precip from knmi_rain_daily.csv

#import soil moisture
SM= pd.read_csv('Soil_moisture.csv')
SM['datetime']=pd.to_datetime(SM['datetime'])
SM.index=SM['datetime']
SM.drop(columns='datetime',inplace=True)
#SM has columns SM-003	SM-020	SM-050	SM-100	'SM_l1' (i.e. 0-50cm)	'SM_l2' (i.e. 50-100)
#and runs from 2005-04-11 00:00:00 2020-12-31 23:30:00 and freq 30 min

#import precipication KNMI
P_k = pd.read_csv('knmi_rain_daily.csv')
P_k['date']=pd.to_datetime(P_k['date'])
P_k.index=P_k.date
P_k.drop(columns=['date','days_since_rain'],inplace=True) #remove date because it's duplicate, remove days_since_rain because it wont be used here
#P_k has columns 'RD' (daily rain from 8pm to 8am next day) and range 2001-01-01 - 2020-12-31 , freq=daily

#import sapflow
Sapf_data=pd.read_csv('Sapflow.csv')
Sapf_data['TIMESTAMP']=pd.to_datetime(Sapf_data['TIMESTAMP'])
Sapf_data.index=Sapf_data['TIMESTAMP']
Sapf_data.drop(columns='TIMESTAMP',inplace=True)
#Sapf_data has columns Jt_1	Jt_2	Jt_3	Jt_4	Jt_5	Jt_6	Jt_mean (i.e. average of all 6)
#and runs from 2012-01-01 00:30:00 - 2015-11-06 19:30:00, freq=30m

### Define plot wrapper for visualizing LE, H, Bowen, Precip, sapflow, soilmost

In [ ]:
# new plot wrapper

plt.rcParams["figure.figsize"] = (20,6)

#data df formats and columns (for my convenience)

#LE_f df_Comb, dont even need ot run filter but scrub negative values or set quality flag limit
#H_f df_Comb, ``
#Bowen (implement in the function)
#GPP (bonus, implement later)
#SM SM['SM_l1'] and SM['SM_l2']
#Precip P_k['RD']
#Sapflow Sapf_data['Jt_mean']


def deep_uptake_plot_wrapper(LE,H,Bowen,SM_l1,SM_l2,Precip,Sapflow,st,ed,heatwave_st=None,heatwave_ed=None,gridarg=None):
    fig,ax = plt.subplots(ncols=1,nrows=2,sharex=True)
    #fig.subplots_adjust(right=0.75)

    #index the input for the timewindow here instead of in the plot function
    LE=LE.loc[st:ed]
    H=H.loc[st:ed]
    Bowen=Bowen.loc[st:ed]
    SM_l1=SM_l1.loc[st:ed]
    SM_l2=SM_l2.loc[st:ed]
    Precip=Precip.loc[st:ed]
    Sapflow=Sapflow.loc[st:ed]

    #create additional y axes for plotting
    #note: ax[0] is the default top y-axis, used for LE
    twin1 = ax[0].twinx() #this one is for Bowen
    #note: ax[1] is the default bottom y-axis, used for Precip
    twin2 = ax[1].twinx() #this one is for SM
    twin3 = ax[1].twinx() #this one is for Sapflow
    
    # Offset the twin axis below the host and label them
    #fig.subplots_adjust(bottom=0.2)
    ax[0].set_ylabel("LE [Wm-2]")
    #ax[0].set_ylim(-200,250)

    twin1.set_ylabel("Bowen ratio [-]")
#    twin1.set_ylim(0,3)

    ax[1].set_ylabel('Daily Precip [mm]')
    
    #twin2.spines["right"].set_position(("axes", 1.05)) #does not need to be pushed right
    twin2.set_ylabel("Soil moist [vol%]")
#    twin2.set_ylim(0,0.1)

    twin3.spines["right"].set_position(("axes", 1.05))
    twin3.set_ylabel("Sapflow [cm s-1]")
#    twin3.set_ylim(0,1)

    
    ax[1].bar(Precip.index,Precip.loc[:,'RD'],width=1, align='edge', color="blue",alpha=0.5, label="KNMI Precip")[0]
    
    ax[0].plot(LE.resample('2H').mean(),c='blue',label="LE")
    twin1.plot(Bowen.resample('2H').mean(),c='red',label="Bowen")
    twin2.plot(SM_l1,c='magenta',label="Soil moisture upper (0-50cm)")
    twin2.plot(SM_l2,c='purple',label="Soil moisture lower (50-100cm)")
    twin3.plot(Sapflow,c='green',label="Sapflow (mean)")        

    if (heatwave_st!=None) & (heatwave_ed != None):
        ax[0].fill_between(x=[heatwave_st,heatwave_ed],y1=0,y2=1, color='orange', alpha=0.3,transform=ax[0].get_xaxis_transform(),label='heatwave (KNMI)')
        ax[1].fill_between(x=[heatwave_st,heatwave_ed],y1=0,y2=1, color='orange', alpha=0.3,transform=ax[1].get_xaxis_transform(),)

#    ax[0].legend(loc=("upper left"))
#    ax[1].legend(loc=("upper left"))
    fig.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2, fancybox=True, shadow=True) #from https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot
    fig.autofmt_xdate()
    if gridarg==None:
        ax[0].grid()
        ax[1].grid()
    else:
        ax.grid(which=gridarg)

    plt.show()

### end of plot wrapper for visualizing LE, H, Bowen, Precip, sapflow, soilmost

# completed special plots for heatwaves 2013 and 2015

In [ ]:
st='2013-07-03'
ed='2013-07-27'
deep_uptake_plot_wrapper(LE_plotting,H_plotting,Bowen_plotting['Bowen'],SM['SM_l1'],SM['SM_l2'],P_k,Sapf_data['Jt_mean'],
                         st,ed,heatwave_st='2013-07-21',heatwave_ed='2013-07-27',gridarg=None)

In [ ]:
st='2015-06-23'
ed='2015-07-05'
deep_uptake_plot_wrapper(LE_plotting,H_plotting,Bowen_plotting['Bowen'],SM['SM_l1'],SM['SM_l2'],P_k,Sapf_data['Jt_mean'],
                         st,ed,heatwave_st='2015-06-30',heatwave_ed='2015-07-05',gridarg=None)

## putting it together to derive fstr

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)

#remove non-growthseason values
#df_tmp1d=dftmp3h.resample('1D').mean()
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=8)]

df_tmp7d['doy']=df_tmp7d.index.dayofyear

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.2]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,1.3)
ax.set_ylabel('LEobs/LEsim')
ax.set_xlabel('Day of Year')

for year in ['2013','2014']:#['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_tmp1d.loc[start:end,'doy'],df_tmp1d.loc[start:end,'LE/ET'], marker='^',label=year)
fig.legend(loc=(0.8715,0.42))
fig.suptitle('LEobs/LEsim for the 3hour timestep, then resample("7days").mean()\n for 09:00-15:00 and no rain in last 3 days ')
ax.grid()